In [1]:
import pandas as pd
import json

In [2]:
#1.Leer el CSV
df = pd.read_csv("../Dataset/movies_dataset.csv", delimiter=',', encoding='utf-8',low_memory=False)

# Desanidado

In [3]:
def parse_json(x):
    """
    Convierte una cadena JSON en un objeto Python (diccionario o lista).
    Maneja valores NaN o valores no cadenas con un manejo de errores básico.
    
    Parámetros:
    x (str): Cadena JSON a convertir.
    
    Retorna:
    dict, list o empty list: Objeto Python deserializado, o una lista vacía si hay un error o no es una cadena.
    """
    if pd.isna(x):  # Manejar valores NaN
        return {}    
    if isinstance(x, str):  # Solo intentar convertir si x es una cadena
        try:
            # Reemplaza comillas simples por dobles para JSON válido
            x = x.replace("'", '"')
            parsed = json.loads(x)
            if isinstance(parsed, (dict, list)):
                return parsed
        except json.JSONDecodeError:
            pass
    return {}  # Retornar un diccionario vacío si es NaN o no es una cadena 
        # [] Retorna una lista vacía para valores no cadenas (NaN, None, números)

In [4]:
def desanidar_columna(df, columna):
    """
    Desanida una columna de un DataFrame que contiene diccionarios o listas de diccionarios.
    
    Parámetros:
    df (pd.DataFrame): El DataFrame que contiene la columna a desanidar.
    columna (str): El nombre de la columna que contiene los datos anidados.
    
    Retorna:
    pd.DataFrame: El DataFrame con la columna desanidada.
    """
    if columna not in df.columns:
        print(f"Columna '{columna}' no encontrada en el DataFrame.")
        return df
    
    try:
        # Aplicamos la función parse_json para asegurarnos de que los datos estén en formato de diccionario o lista
        df[columna] = df[columna].apply(parse_json)
        
        if isinstance(df[columna].iloc[0], list):
            # Desanidamos la columna si es una lista de diccionarios
            df_exploded = df.explode(columna).reset_index(drop=True)
            df_expanded = pd.json_normalize(df_exploded[columna])
            df_expanded.columns = [f"{columna}_{col}" for col in df_expanded.columns]
        elif isinstance(df[columna].iloc[0], dict):
            # Expandimos los diccionarios si no es una lista
            df_expanded = pd.json_normalize(df[columna])
            df_expanded.columns = [f"{columna}_{col}" for col in df_expanded.columns]
        else:
            # Si no es ni lista ni diccionario, retornamos el DataFrame sin cambios
            print(f"La columna '{columna}' no contiene datos válidos para desanidar.")
            return df
        
        # Concatenamos las nuevas columnas al DataFrame original (sin duplicar la columna original desanidada)
        df_final = pd.concat([df.drop(columns=[columna]), df_expanded], axis=1)
        #df_final = pd.concat([df.drop(columns=[columna]), df_expanded], axis=1, join='inner')
        return df_final
    except Exception as e:
        print(f"Error al desanidar la columna '{columna}': {e}")
        return df
    
        

In [7]:
#2. Desanidar los datos

# Lista de columnas que contienen datos JSON
columnas_json = [
    'belongs_to_collection',
    'genres',
    'production_companies',
    'production_countries',
    'spoken_languages'    
]

# Desanidar las columnas
for columna in columnas_json:
    df = desanidar_columna(df, columna)



Columna 'belongs_to_collection' no encontrada en el DataFrame.
Columna 'genres' no encontrada en el DataFrame.
Columna 'production_companies' no encontrada en el DataFrame.
Columna 'production_countries' no encontrada en el DataFrame.
Columna 'spoken_languages' no encontrada en el DataFrame.


In [8]:
print(df.shape)

(150807, 31)


In [9]:
print(df.columns.tolist())

['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'belongs_to_collection_id', 'belongs_to_collection_name', 'belongs_to_collection_poster_path', 'belongs_to_collection_backdrop_path', 'genres_id', 'genres_name', 'production_companies_name', 'production_companies_id', 'production_countries_iso_3166_1', 'production_countries_name', 'spoken_languages_iso_639_1', 'spoken_languages_name']


In [10]:
print(df['revenue'].head())

0    373554033.0
1    262797249.0
2            0.0
3     81452156.0
4     76578911.0
Name: revenue, dtype: float64


# Transformaciones

### Los valores nulos de los campos 'revenue', 'budget' deben ser rellenados por el número 0

In [11]:
# Contar los valores nulos en las columnas 'revenue' y 'budget'
nulos_revenue = df['revenue'].isnull().sum()
nulos_budget = df['budget'].isnull().sum()

print(f"Valores nulos en 'revenue': {nulos_revenue}")
print(f"Valores nulos en 'budget': {nulos_budget}")

Valores nulos en 'revenue': 105347
Valores nulos en 'budget': 105341


In [12]:
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Verificar si quedan valores nulos en estas columnas
print(df[['revenue', 'budget']].isnull().sum())

revenue    0
budget     0
dtype: int64


### Los valores nulos del campo 'release_date' deben eliminarse.

In [13]:
# Contar los valores nulos en la columna 'release_date' antes de la transformación
nulos_release_date_antes = df['release_date'].isnull().sum()

# Contar el número de registros antes de la eliminación
registros_antes = len(df)

print(f"Valores nulos en 'release_date' antes de eliminar: {nulos_release_date_antes}")
print(f"Total de registros antes de eliminar: {registros_antes}")


Valores nulos en 'release_date' antes de eliminar: 105428
Total de registros antes de eliminar: 150807


In [14]:
# Eliminar filas con valores nulos en la columna 'release_date'
df = df.dropna(subset=['release_date'])

In [15]:
# Contar los valores nulos en la columna 'release_date' después de la transformación
nulos_release_date_despues = df['release_date'].isnull().sum()

# Contar el número de registros después de la eliminación
registros_despues = len(df)

print(f"Valores nulos en 'release_date' después de eliminar: {nulos_release_date_despues}")
print(f"Total de registros después de eliminar: {registros_despues}")

Valores nulos en 'release_date' después de eliminar: 0
Total de registros después de eliminar: 45379


### De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [16]:
# Convertir la columna 'release_date' a formato datetime, manejando errores
print("Antes de la conversión:")
print(df['release_date'].head())

df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

# Verificar las primeras filas de la columna 'release_date' para confirmar la conversión
print("Después de la conversión:")
print(df['release_date'].head())

Antes de la conversión:
0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object
Después de la conversión:
0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]


In [17]:
# Contar los valores nulos en la columna 'release_date' antes de la conversión
nulos_release_date_antes = df['release_date'].isnull().sum()
print(f"Valores nulos en 'release_date' antes de la conversión: {nulos_release_date_antes}")

# Crear una nueva columna 'release_year' que contenga solo el año de 'release_date'
df['release_year'] = df['release_date'].dt.year

# Convertir 'release_year' a entero, manejando los NaN (valores nulos)
df['release_year'] = df['release_year'].fillna(0).astype(int)

# Contar los valores nulos en la columna 'release_year' después de la conversión
nulos_release_year_despues = df['release_year'].isnull().sum()

# Mostrar los valores nulos en la columna 'release_year' después de la conversión
print(f"Valores nulos en 'release_year' después de la conversión: {nulos_release_year_despues}")

# Verificar las primeras filas de la nueva columna 'release_year'
print(df[['release_date', 'release_year']].head())

Valores nulos en 'release_date' antes de la conversión: 3
Valores nulos en 'release_year' después de la conversión: 0
  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995
3   1995-12-22          1995
4   1995-02-10          1995


### Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [18]:
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Verificar los tipos de datos para confirmar la conversión
#print(df.dtypes)

In [19]:
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)
print(df[['revenue', 'budget', 'return']].head())

       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0   0.000000


### Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [20]:
columnas_antes = len(df.columns)
print(f"Número de columnas antes de eliminar: {columnas_antes}")

# Eliminar las columnas no deseadas
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df = df.drop(columns=columnas_a_eliminar)

columnas_despues = len(df.columns)
print(f"Número de columnas después de eliminar: {columnas_despues}")

Número de columnas antes de eliminar: 33
Número de columnas después de eliminar: 27


In [21]:
print(df.columns.tolist())

['budget', 'id', 'original_language', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'belongs_to_collection_id', 'belongs_to_collection_name', 'belongs_to_collection_poster_path', 'belongs_to_collection_backdrop_path', 'genres_id', 'genres_name', 'production_companies_name', 'production_companies_id', 'production_countries_iso_3166_1', 'production_countries_name', 'spoken_languages_iso_639_1', 'spoken_languages_name', 'release_year', 'return']


# Almacenamiento en formato parquet

In [23]:
try:
    # Guardar el DataFrame en formato Parquet usando pyarrow
    df.to_parquet("../Dataset/movies_dataset.parquet", engine='pyarrow')
    print("El DataFrame ha sido guardado en formato Parquet exitosamente.")
except Exception as e:
    # Manejar cualquier excepción que ocurra durante la operación
    print(f"Ocurrió un error al intentar guardar el DataFrame en formato Parquet: {e}")

El DataFrame ha sido guardado en formato Parquet exitosamente.
